## 다음 문제를 풀이하시오.

diabetes.csv 파일을 대상으로 다음 문제를 풀이하시오.

In [1]:
import pandas as pd
df = pd.read_csv("diabetes.csv")
df.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

### Q1. 당뇨 여부인 Outcome과 임신 횟수인 Pregnancies을 제외한 수치형 변수는  결측치가 0으로 기록이 되어 있다. 모든 변수의 결측치 개수를 더한 개수를 기술하시오.
(정답 예시: 123)

In [2]:
sum(df["Glucose"] == 0)
sum(df["BloodPressure"] == 0)
sum(df["SkinThickness"] == 0)

227

In [3]:
df.iloc[:, 1:-1].apply(lambda x: sum(x == 0))

Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64

In [4]:
df.iloc[:, 1:-1].apply(lambda x: sum(x == 0)).sum()

652

### Q2. Outcome 변수를 제외한 변수간 상관분석을 실시하고 상관계수가 가장 높은 조합의 변수명과 상관계수를 기술하시오.
 - 변수명은 알파벳 순서대로 기술한다.
 - 상관계수는 반올림하여 소수점 셋째자리까지 기술한다.
 
(정답 예시: aaa, bbb, 0.123)

In [5]:
df.drop("Outcome", axis = 1).corr()

Pregnancies   Glucose  BloodPressure  SkinThickness  \
Pregnancies                  1.000000  0.129459       0.141282      -0.081672   
Glucose                      0.129459  1.000000       0.152590       0.057328   
BloodPressure                0.141282  0.152590       1.000000       0.207371   
SkinThickness               -0.081672  0.057328       0.207371       1.000000   
Insulin                     -0.073535  0.331357       0.088933       0.436783   
BMI                          0.017683  0.221071       0.281805       0.392573   
DiabetesPedigreeFunction    -0.033523  0.137337       0.041265       0.183928   
Age                          0.544341  0.263514       0.239528      -0.113970   

                           Insulin       BMI  DiabetesPedigreeFunction  \
Pregnancies              -0.073535  0.017683                 -0.033523   
Glucose                   0.331357  0.221071                  0.137337   
BloodPressure             0.088933  0.281805                  0.041265   
SkinThickness             0.436783  0.392573                  0.183928   
Insulin                   1.000000  0.197859                  0.185071   
BMI                       0.197859  1.000000                  0.140647   
DiabetesPedigreeFunction  0.185071  0.140647                  1.000000   
Age                      -0.042163  0.036242                  0.033561   

                               Age  
Pregnancies               0.544341  
Glucose                   0.263514  
BloodPressure             0.239528  
SkinThickness            -0.113970  
Insulin                  -0.042163  
BMI                       0.036242  
DiabetesPedigreeFunction  0.033561  
Age                       1.000000

In [6]:
df.drop("Outcome", axis = 1).corr().apply(lambda x: x[x != 1].max())

Pregnancies                 0.544341
Glucose                     0.331357
BloodPressure               0.281805
SkinThickness               0.436783
Insulin                     0.436783
BMI                         0.392573
DiabetesPedigreeFunction    0.185071
Age                         0.544341
dtype: float64

In [7]:
ser = df.drop("Outcome", axis = 1).corr().apply(lambda x: x[x != 1].max())
ser[ser == ser.max()]

Pregnancies    0.544341
Age            0.544341
dtype: float64

In [8]:
ser.idxmax()

'Pregnancies'

### Q3. 임신 경험 여부와 BMI간의 관계를 보고자 한다. 이때 적절한 검정방법을 사용하고 해당 검정의 검정통계량의 절대값을 반올림하여 소수점 셋째 자리 까지 표기하시오.
(정답 예시: 1.234)

In [9]:
df.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

In [10]:
df["Pregnancies"].value_counts()

1     135
0     111
2     103
3      75
4      68
5      57
6      50
7      45
8      38
9      28
10     24
11     11
13     10
12      9
14      2
15      1
17      1
Name: Pregnancies, dtype: int64

In [11]:
from scipy.stats.stats import ttest_ind

In [12]:
stat, p = ttest_ind(df.loc[df["Pregnancies"] != 0, "BMI"],
                    df.loc[df["Pregnancies"] == 0, "BMI"])
round(abs(stat), 3)

3.341

### Q4. 임신 횟수, 혈당, 혈압, 피부 두께로 BMI를 설명할 수 있는지 확인해보려 한다. BMI를 종속변수로 한 다중선형 회귀분석을 실시하고 유의미한 변수의 개수와 결정계수를 기술하시오.
 - 결측치는 모두 제거하고 분석을 실시한다.
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 1, 0.12)

In [13]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [14]:
# df_q4 = df[["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "BMI"]]
df_q4 = df.iloc[:, [0, 1, 2, 3, 5]]
df_q4.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [15]:
import numpy as np
ser = pd.Series([1, 2, 3])
ser.replace(2, np.nan).dropna()

0    1.0
2    3.0
dtype: float64

In [16]:
df_q4_notna = df_q4.replace(0, np.nan).dropna()
df_q4_notna.head(1)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0          6.0    148.0           72.0           35.0  33.6

In [17]:
print(len(df_q4))
print(len(df_q4_notna))

768
455


In [18]:
sum(df_q4["BMI"] == 0)

11

In [19]:
for col in df_q4.columns[1:]:
    print(col)
    df_q4 = df_q4.loc[df_q4[col] != 0, ]

Glucose
BloodPressure
SkinThickness
BMI


In [20]:
print(len(df))
print(len(df_q4))

768
532


In [21]:
df_q4.head()

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6
3            1       89             66             23  28.1
4            0      137             40             35  43.1
6            3       78             50             32  31.0

In [22]:
from statsmodels.formula.api import ols
formula = "BMI ~ " + "+".join(df_q4.columns[:-1])
formula

'BMI ~ Pregnancies+Glucose+BloodPressure+SkinThickness'

In [23]:
model = ols(formula, df_q4).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BMI   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     112.5
Date:                Fri, 26 Jun 2020   Prob (F-statistic):           2.86e-69
Time:                        10:22:44   Log-Likelihood:                -1616.3
No. Observations:                 532   AIC:                             3243.
Df Residuals:                     527   BIC:                             3264.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        12.9673      1.447      8.962      0.000      10.125      15.810
Pregnancies      -0.1960      0.068     -2.875      0.004      -0.330      -0.062
Glucose           0.0190      0.007      2.556      0.011       0.004       0.034
BloodPressure     0.0966      0.019      5.097      0.000       0.059       0.134
SkinThickness     0.3910      0.022     17.875      0.000       0.348       0.434
==============================================================================
Omnibus:                       42.530   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.189
Skew:                           0.304   Prob(JB):                     9.83e-31
Kurtosis:                       5.422   Cond. No.                         964.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
model.rsquared

0.4605519170833461

In [25]:
round(model.rsquared, 2)

0.46

### Q5. 결측치 처리에 따른 효과를 알아보고자 한다. 4번에서 사용된 독립변수 중 결측치를 모두 해당 변수의 평균치로 치환하고 다중선형 회귀분석을 실시하여 얻은 결정계수와 결측치를 연령대 평균 값으로 치환한 후 다중선형 회귀분석을 실시하여 얻은 결정계수를 차례대로 기술하시오.
 - 단, 이 분석은 70세 미만의 데이터만 사용한다.
 - 연령대 구분 단위는 10살 단위로 한다(17 => 1, 30 => 3, 45 => 4)
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 0.12, 0.34)

In [26]:
df_q5_1 = df.loc[df["Age"] < 70, ].copy()
df_q5_2 = df.loc[df["Age"] < 70, ].copy()

In [27]:
df_q5_1 = df_q5_1.iloc[:, [0, 1, 2, 3, 5]]
df_q5_1.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [28]:
print(len(df_q5_1)) # 3개 row 제거됨
print(len(df))

765
768


In [29]:
df_q5_1.mean()

Pregnancies        3.840523
Glucose          120.848366
BloodPressure     69.172549
SkinThickness     20.550327
BMI               32.016078
dtype: float64

In [30]:
for col in df_q5_1.columns[1:]:
    df_q5_1.loc[df_q5_1[col] == 0, col] = df_q5_1.loc[df_q5_1[col] != 0, col].mean()
df_q5_1.mean()

Pregnancies        3.840523
Glucose          121.643421
BloodPressure     72.389877
SkinThickness     29.166976
BMI               32.483156
dtype: float64

In [31]:
formula = "BMI ~ " + "+".join(df_q5_1.columns[:-1])
model_q5_1 = ols(formula, df_q5_1).fit()
round(model_q5_1.rsquared, 3)

0.343

In [ ]:
df_q5_2.head(2)

In [34]:
# pd.Series([7, 15, 30]) // 10

In [35]:
df_q5_2["Age_group"] = df_q5_2["Age"] // 10
df_q5_2.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  Age_group  
0                     0.627   50        1          5  
1                     0.351   31        0          3

In [36]:
df_q5_2["Age_group"].value_counts()

2    396
3    165
4    118
5     57
6     29
Name: Age_group, dtype: int64

In [ ]:
for col in df_q5_2.columns[1:]:
    for age_group in df_q5_2["Age_group"].unique():
        cond1 = df_q5_2["Age_group"] == age_group # 조건 1
        cond2 = df_q5_2[col] == 0
        col_mean = df_q5_2.loc[cond1 & (df_q5_2[col] != 0), col].mean()
        df_q5_2.loc[cond1 & cond2, col] = col_mean
df_q5_2.head(2)

In [ ]:
df_q5_2.mean()

In [ ]:
df_q5_2_agg = df_q5_2.groupby("Age_group").agg(lambda x: x[x != 0].mean())
df_q5_2_agg

In [ ]:
df_q5_2_agg.columns = "mean_" + df_q5_2_agg.columns.values
df_q5_2_agg.head(2)

In [ ]:
df_q5_2 = df_q5_2.iloc[:, [0, 1, 2, 3, 5]]
df_q5_2.head(2)

In [47]:
formula = "BMI ~ " + "+".join(df_q5_2.columns[:-1])
model_q5_2 = ols(formula, df_q5_2).fit()
round(model_q5_2.rsquared, 3)

0.337

In [48]:
model_q5_1.rsquared - model_q5_2.rsquared

0.006069340508791821

In [49]:
test = df.copy()
test.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

In [51]:
test.add_suffix("_mean").head(2)

Pregnancies_mean  Glucose_mean  BloodPressure_mean  SkinThickness_mean  \
0                 6           148                  72                  35   
1                 1            85                  66                  29   

   Insulin_mean  BMI_mean  DiabetesPedigreeFunction_mean  Age_mean  \
0             0      33.6                          0.627        50   
1             0      26.6                          0.351        31   

   Outcome_mean  
0             1  
1             0

In [52]:
test = test.add_suffix("_mean")
test.head(2)

Pregnancies_mean  Glucose_mean  BloodPressure_mean  SkinThickness_mean  \
0                 6           148                  72                  35   
1                 1            85                  66                  29   

   Insulin_mean  BMI_mean  DiabetesPedigreeFunction_mean  Age_mean  \
0             0      33.6                          0.627        50   
1             0      26.6                          0.351        31   

   Outcome_mean  
0             1  
1             0

### Q6. 분류모델의 성능을 비교하고자 한다. Outcome 변수를 독립변수로 하고 나머지 변수를 독립변수로 하고, 독립변수의 경우 결측치로 간주되는 것은 모두 제거한다. 이항 로지스틱 회귀분석과, 의사결정나무로 학습을 하고 그 정확도를 각각 기술하시오.
 - 학습은 최초 100개 데이터를 제외한 나머지로 실시한다.
 - 평가는 최초 100개 데이터로 실시한다.
 - 이항 로지스틱 회귀분석의 경우 유의미 하지 않은 변수는 제거한다.
 - 이항 로지스틱 회귀분석의 문턱값은 0.5로 설정한다.
 - 정확도는 반올림하여 소수점 셋째 자리까지 표기한다.
 
(정답 예시: 0.123, 0.456)

In [54]:
from statsmodels.api import Logit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [55]:
df_q6 = df.copy()
df_q6.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

In [58]:
df_q6_notna = df_q6.drop(["Pregnancies", "Outcome"], axis = 1).replace(0, np.nan).dropna()
print(len(df_q6_notna))
df_q6_notna.head(2)

392


Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
3     89.0           66.0           23.0     94.0  28.1   
4    137.0           40.0           35.0    168.0  43.1   

   DiabetesPedigreeFunction  Age  
3                     0.167   21  
4                     2.288   33

In [60]:
df_q6_notna = df_q6_notna.join(df_q6[["Pregnancies", "Outcome"]], how = "inner")
df_q6_notna.head(2)

Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
3     89.0           66.0           23.0     94.0  28.1   
4    137.0           40.0           35.0    168.0  43.1   

   DiabetesPedigreeFunction  Age  Pregnancies  Outcome  
3                     0.167   21            1        0  
4                     2.288   33            0        1

In [69]:
train_q6 = df_q6_notna.iloc[100:, ].copy()
test_q6  = df_q6_notna.iloc[:100, ].copy()
print(len(train_q6))
print(len(test_q6))

292
100


In [70]:
model_LR = Logit(train_q6["Outcome"], 
                 train_q6.drop("Outcome", axis = 1)).fit()
model_LR.summary()

Optimization terminated successfully.
         Current function value: 0.538995
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  292
Model:                          Logit   Df Residuals:                      284
Method:                           MLE   Df Model:                            7
Date:                Fri, 26 Jun 2020   Pseudo R-squ.:                  0.1234
Time:                        11:18:32   Log-Likelihood:                -157.39
converged:                       True   LL-Null:                       -179.54
Covariance Type:            nonrobust   LLR p-value:                 1.859e-07
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Glucose                      0.0220      0.006      3.897      0.000       0.011       0.033
BloodPressure               -0.0505      0.012     -4.192      0.000      -0.074      -0.027
SkinThickness                0.0401      0.018      2.241      0.025       0.005       0.075
Insulin                      0.0018      0.001      1.323      0.186      -0.001       0.005
BMI                         -0.0594      0.028     -2.137      0.033      -0.114      -0.005
DiabetesPedigreeFunction     0.4404      0.409      1.077      0.282      -0.361       1.242
Age                         -0.0037      0.019     -0.194      0.846      -0.042       0.034
Pregnancies                  0.1348      0.059      2.280      0.023       0.019       0.251
============================================================================================
"""

In [79]:
# model_LR = Logit(train_q6["Outcome"], 
#                  train_q6.drop(["Outcome", "Age",
#                                 "DiabetesPedigreeFunction",
#                                 "Insulin", "BMI"], axis = 1)).fit()
model_LR = Logit(train_q6["Outcome"], 
                 train_q6[["Glucose", "BloodPressure", "Pregnancies"]]).fit()
model_LR.summary()

Optimization terminated successfully.
         Current function value: 0.553627
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  292
Model:                          Logit   Df Residuals:                      289
Method:                           MLE   Df Model:                            2
Date:                Fri, 26 Jun 2020   Pseudo R-squ.:                 0.09961
Time:                        11:22:14   Log-Likelihood:                -161.66
converged:                       True   LL-Null:                       -179.54
Covariance Type:            nonrobust   LLR p-value:                 1.711e-08
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Glucose           0.0234      0.005      5.180      0.000       0.015       0.032
BloodPressure    -0.0587      0.009     -6.631      0.000      -0.076      -0.041
Pregnancies       0.1479      0.043      3.415      0.001       0.063       0.233
=================================================================================
"""

In [81]:
pred_LR = model_LR.predict(test_q6[["Glucose", "BloodPressure", "Pregnancies"]])
pred_LR_class = (pred_LR > 0.5) + 0 # 0.5는 문턱값
pred_LR_class.value_counts()

0    70
1    30
dtype: int64

In [82]:
pd.crosstab(test_q6["Outcome"], pred_LR_class)

col_0     0   1
Outcome        
0        48  11
1        22  19

In [90]:
model_DC = DecisionTreeClassifier().fit(y = train_q6["Outcome"], 
                                        X = train_q6.drop("Outcome", axis = 1))

In [93]:
pred_DC = model_DC.predict(test_q6.drop("Outcome", axis = 1))
pd.crosstab(test_q6["Outcome"], pred_DC)

col_0     0   1
Outcome        
0        48  11
1        23  18

In [94]:
print(accuracy_score(test_q6["Outcome"], pred_LR_class))
print(accuracy_score(test_q6["Outcome"], pred_DC))

0.67
0.66
